## Car Loan Default Prediction

Given data about vehicle loans, let's try to predict if a given loanee will default or not. 

We will use logistic regression model to make our predictions.

data source: https://www.kaggle.com/datasets/mamtadhaker/lt-vehicle-loan-default-prediction?resource=download&select=train.csv

### Getting Started

In [59]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import re
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings(action='ignore')

In [60]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [61]:
train_df

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,03-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,26-09-18,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,01-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,26-10-18,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,26-09-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233149,626432,63213,105405,60.72,34,20700,48,1050,01-08-88,Salaried,26-10-18,6,3705,1,0,0,1,0,0,735,D-Very Low Risk,4,3,0,390443,416133,416133,0,0,0,0,0,0,4084,0,0,0,1yrs 9mon,3yrs 3mon,0,0
233150,606141,73651,100600,74.95,34,23775,51,990,05-12-88,Self employed,23-10-18,6,3705,1,0,0,1,0,0,825,A-Very Low Risk,1,0,0,0,0,0,0,0,0,0,0,0,1565,0,0,0,0yrs 6mon,0yrs 6mon,0,0
233151,613658,33484,71212,48.45,77,22186,86,2299,01-06-76,Salaried,24-10-18,4,3479,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
233152,548084,34259,73286,49.10,77,22186,86,2299,26-03-94,Salaried,29-09-18,4,3479,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0


In [62]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 41 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UniqueID                             233154 non-null  int64  
 1   disbursed_amount                     233154 non-null  int64  
 2   asset_cost                           233154 non-null  int64  
 3   ltv                                  233154 non-null  float64
 4   branch_id                            233154 non-null  int64  
 5   supplier_id                          233154 non-null  int64  
 6   manufacturer_id                      233154 non-null  int64  
 7   Current_pincode_ID                   233154 non-null  int64  
 8   Date.of.Birth                        233154 non-null  object 
 9   Employment.Type                      225493 non-null  object 
 10  DisbursalDate                        233154 non-null  object 
 11  State_ID     

In [63]:
test_df

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
0,655269,53478,63558,86.54,67,22807,45,1497,01-01-74,Salaried,03-11-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
1,723482,55513,63163,89.45,67,22807,45,1497,20-05-85,Self employed,20-11-18,6,1998,1,1,0,0,0,0,749,C-Very Low Risk,2,1,0,43898,48780,48780,0,0,0,0,0,0,5605,0,1,0,0yrs 8mon,1yrs 0mon,1
2,758529,65282,84320,79.93,78,23135,86,2071,14-10-95,Salaried,29-11-18,4,1646,1,1,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
3,763449,46905,63896,76.58,78,17014,45,2070,01-06-73,Self employed,29-11-18,4,1646,1,1,0,0,0,0,14,Not Scored: Only a Guarantor,1,1,1,132480,255000,255000,0,0,0,0,0,0,0,0,0,0,2yrs 5mon,2yrs 5mon,0
4,708663,51428,63896,86.08,78,17014,45,2069,01-06-72,Salaried,17-11-18,4,1646,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112387,755547,44875,72509,67.58,74,16646,86,2591,14-05-92,Self employed,28-11-18,4,3823,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
112388,716994,53549,72509,78.61,74,16646,86,2593,24-04-84,Self employed,19-11-18,4,3823,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
112389,709352,55243,72509,81.37,74,16646,86,2593,01-01-86,Self employed,17-11-18,4,3823,1,1,0,0,0,0,785,B-Very Low Risk,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4yrs 10mon,9yrs 5mon,0
112390,699795,55243,72509,81.37,74,16646,86,2593,01-01-73,Self employed,15-11-18,4,3823,1,1,0,0,0,0,714,D-Very Low Risk,1,1,0,36380,100000,100000,0,0,0,0,0,0,0,0,1,1,0yrs 3mon,0yrs 3mon,0


In [64]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112392 entries, 0 to 112391
Data columns (total 40 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UniqueID                             112392 non-null  int64  
 1   disbursed_amount                     112392 non-null  int64  
 2   asset_cost                           112392 non-null  int64  
 3   ltv                                  112392 non-null  float64
 4   branch_id                            112392 non-null  int64  
 5   supplier_id                          112392 non-null  int64  
 6   manufacturer_id                      112392 non-null  int64  
 7   Current_pincode_ID                   112392 non-null  int64  
 8   Date.of.Birth                        112392 non-null  object 
 9   Employment.Type                      108949 non-null  object 
 10  DisbursalDate                        112392 non-null  object 
 11  State_ID     

### Preprocessing

In [65]:
def encode_dates(df, column):
    df = df.copy()
    df[column] = pd.to_datetime(df[column], format='%d-%m-%y')
    df[column + '_year'] = df[column].apply(lambda x: x.year)
    df[column + '_month'] = df[column].apply(lambda x: x.month)
    df[column + '_day'] = df[column].apply(lambda x: x.day)
    df = df.drop(column, axis=1)
    return df

def encode_durations(df, column):
    df = df.copy()
    df[column + '_yrs'] = df[column].apply(lambda x: re.search(r'\d+(?=yrs)', x).group(0)).astype(int)
    df[column + '_mon'] = df[column].apply(lambda x: re.search(r'\d+(?=mon)', x).group(0)).astype(int)
    df = df.drop(column, axis=1)
    return df

def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [80]:
def preprocess_inputs(df):
    df = df.copy()

    # Drop UniqueID and MobileNo_Avl_Flag columns
    df = df.drop(['UniqueID', 'MobileNo_Avl_Flag'], axis=1)

    # Drop high-cardinality columns
    df = df.drop(['supplier_id', 'Current_pincode_ID', 'Employee_code_ID'], axis=1)
    
    # Encode missing employment values as Unemployed
    df['Employment.Type'] = df['Employment.Type'].fillna('Unemployed')

    # Extract date features 
    df = encode_dates(df, column='Date.of.Birth')
    df = encode_dates(df, column='DisbursalDate')

    # Drop disbursaldate_year
    df = df.drop('DisbursalDate_year', axis=1)

    # Extract duration features
    df = encode_durations(df, column='AVERAGE.ACCT.AGE')
    df = encode_durations(df, column='CREDIT.HISTORY.LENGTH')

    # One hot encode categorical features
    for column in ['branch_id', 'manufacturer_id', 'Employment.Type', 'State_ID', 'PERFORM_CNS.SCORE.DESCRIPTION']:
        df = onehot_encode(df, column)
    
    return df

In [81]:
X_train = preprocess_inputs(train_df)
X_test = preprocess_inputs(test_df)

In [68]:
X_train

,disbursed_amount,asset_cost,ltv,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,loan_default,Date.of.Birth_year,Date.of.Birth_month,Date.of.Birth_day,DisbursalDate_year,DisbursalDate_month,DisbursalDate_day,AVERAGE.ACCT.AGE_yrs,AVERAGE.ACCT.AGE_mon,CREDIT.HISTORY.LENGTH_yrs,CREDIT.HISTORY.LENGTH_mon,branch_id_1,branch_id_2,branch_id_3,branch_id_5,branch_id_7,branch_id_8,branch_id_9,branch_id_10,branch_id_11,branch_id_13,branch_id_14,branch_id_15,branch_id_16,branch_id_17,branch_id_18,branch_id_19,branch_id_20,branch_id_29,branch_id_34,branch_id_35,branch_id_36,branch_id_42,branch_id_43,branch_id_48,branch_id_61,branch_id_62,branch_id_63,branch_id_64,branch_id_65,branch_id_66,branch_id_67,branch_id_68,branch_id_69,branch_id_70,branch_id_72,branch_id_73,branch_id_74,branch_id_76,branch_id_77,branch_id_78,branch_id_79,branch_id_82,branch_id_84,branch_id_85,branch_id_97,branch_id_100,branch_id_101,branch_id_103,branch_id_104,branch_id_105,branch_id_111,branch_id_117,branch_id_120,branch_id_121,branch_id_130,branch_id_135,branch_id_136,branch_id_138,branch_id_142,branch_id_146,branch_id_147,branch_id_152,branch_id_153,branch_id_158,branch_id_159,branch_id_160,branch_id_162,branch_id_165,branch_id_202,branch_id_207,branch_id_217,branch_id_248,branch_id_249,branch_id_250,branch_id_251,branch_id_254,branch_id_255,branch_id_257,branch_id_258,branch_id_259,branch_id_260,branch_id_261,manufacturer_id_45,manufacturer_id_48,manufacturer_id_49,manufacturer_id_51,manufacturer_id_67,manufacturer_id_86,manufacturer_id_120,manufacturer_id_145,manufacturer_id_152,manufacturer_id_153,manufacturer_id_156,Employment.Type_Salaried,Employment.Type_Self employed,Employment.Type_Unemployed,State_ID_1,State_ID_2,State_ID_3,State_ID_4,State_ID_5,State_ID_6,State_ID_7,State_ID_8,State_ID_9,State_ID_10,State_ID_11,State_ID_12,State_ID_13,State_ID_14,State_ID_15,State_ID_16,State_ID_17,State_ID_18,State_ID_19,State_ID_20,State_ID_21,State_ID_22,PERFORM_CNS.SCORE.DESCRIPTION_A-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_B-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_C-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_D-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_E-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_F-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_G-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_H-Medium Risk,PERFORM_CNS.SCORE.DESCRIPTION_I-Medium Risk,PERFORM_CNS.SCORE.DESCRIPTION_J-High Risk,PERFORM_CNS.SCORE.DESCRIPTION_K-High Risk,PERFORM_CNS.SCORE.DESCRIPTION_L-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_M-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_No Bureau History Available,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: More than 50 active Accounts found,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Activity seen on the customer (Inactive),PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Updates available in last 36 months,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Not Enough Info available on the customer,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Only a Guarantor,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Sufficient History Not Available
0,50578,58400,89.55,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1984,1,1,2018,8,3,0,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [69]:
X_test

,disbursed_amount,asset_cost,ltv,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,Date.of.Birth_year,Date.of.Birth_month,Date.of.Birth_day,DisbursalDate_year,DisbursalDate_month,DisbursalDate_day,AVERAGE.ACCT.AGE_yrs,AVERAGE.ACCT.AGE_mon,CREDIT.HISTORY.LENGTH_yrs,CREDIT.HISTORY.LENGTH_mon,branch_id_1,branch_id_2,branch_id_3,branch_id_5,branch_id_7,branch_id_8,branch_id_9,branch_id_10,branch_id_11,branch_id_13,branch_id_14,branch_id_15,branch_id_16,branch_id_17,branch_id_18,branch_id_19,branch_id_20,branch_id_29,branch_id_34,branch_id_35,branch_id_36,branch_id_42,branch_id_43,branch_id_48,branch_id_61,branch_id_62,branch_id_63,branch_id_64,branch_id_65,branch_id_66,branch_id_67,branch_id_68,branch_id_69,branch_id_70,branch_id_72,branch_id_73,branch_id_74,branch_id_76,branch_id_77,branch_id_78,branch_id_79,branch_id_82,branch_id_84,branch_id_85,branch_id_97,branch_id_100,branch_id_101,branch_id_103,branch_id_104,branch_id_105,branch_id_111,branch_id_117,branch_id_120,branch_id_121,branch_id_130,branch_id_135,branch_id_136,branch_id_138,branch_id_142,branch_id_146,branch_id_147,branch_id_152,branch_id_153,branch_id_158,branch_id_159,branch_id_160,branch_id_162,branch_id_165,branch_id_202,branch_id_207,branch_id_217,branch_id_248,branch_id_249,branch_id_250,branch_id_251,branch_id_254,branch_id_255,branch_id_257,branch_id_258,branch_id_259,branch_id_260,branch_id_261,manufacturer_id_45,manufacturer_id_48,manufacturer_id_49,manufacturer_id_51,manufacturer_id_67,manufacturer_id_86,manufacturer_id_120,manufacturer_id_145,manufacturer_id_152,manufacturer_id_153,manufacturer_id_155,Employment.Type_Salaried,Employment.Type_Self employed,Employment.Type_Unemployed,State_ID_1,State_ID_2,State_ID_3,State_ID_4,State_ID_5,State_ID_6,State_ID_7,State_ID_8,State_ID_9,State_ID_10,State_ID_11,State_ID_12,State_ID_13,State_ID_14,State_ID_15,State_ID_16,State_ID_17,State_ID_18,State_ID_19,State_ID_20,State_ID_21,State_ID_22,PERFORM_CNS.SCORE.DESCRIPTION_A-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_B-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_C-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_D-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_E-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_F-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_G-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_H-Medium Risk,PERFORM_CNS.SCORE.DESCRIPTION_I-Medium Risk,PERFORM_CNS.SCORE.DESCRIPTION_J-High Risk,PERFORM_CNS.SCORE.DESCRIPTION_K-High Risk,PERFORM_CNS.SCORE.DESCRIPTION_L-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_M-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_No Bureau History Available,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Activity seen on the customer (Inactive),PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Updates available in last 36 months,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Not Enough Info available on the customer,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Only a Guarantor,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Sufficient History Not Available
0,53478,63558,86.54,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1974,1,1,2018,11,3,0,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,Fa

### Adjust X_test dummy columns to match X_train

We can see that after one-hot encoding, there are some columns present in X_train that aren't present in X_test, and vice versa. 
We can correct this with a simple function.

In [82]:
y_train = train_df['loan_default']
X_train = X_train.drop('loan_default', axis=1)

In [83]:
for column in X_train.columns:
    if column not in X_test.columns:
        print(column)

manufacturer_id_156
PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: More than 50 active Accounts found


In [84]:
for column in X_test.columns:
    if column not in X_train.columns:
        print(column)

manufacturer_id_155


In [85]:
def adjust_columns(X_train, X_test):
    train_columns = []
    test_columns = []

    # Get X_train columns not present in X_test
    for column in X_train.columns:
        if column not in X_test.columns:
            train_columns.append(column)

    # # Get X_test columns not present in X_train
    for column in X_test.columns:
        if column not in X_train.columns:
            test_columns.append(column)

    # Remove test columns from X_test
    X_test = X_test.copy()
    X_test = X_test.drop(test_columns, axis=1)

    # Add train columns and fill them with 0s
    for column in train_columns:
        column_index = list(X_train.columns).index(column)
        X_test.insert(loc=column_index, column=column, value=0)

    return X_test

In [86]:
X_test = adjust_columns(X_train, X_test)

In [87]:
(X_train.columns == X_test.columns).all()

np.True_

In [97]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns) 

In [98]:
for column in X_train.columns:
    if len(X_train[column].unique()) == 1:
        print(column)

In [99]:
X_train.describe()

,disbursed_amount,asset_cost,ltv,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,Date.of.Birth_year,Date.of.Birth_month,Date.of.Birth_day,DisbursalDate_month,DisbursalDate_day,AVERAGE.ACCT.AGE_yrs,AVERAGE.ACCT.AGE_mon,CREDIT.HISTORY.LENGTH_yrs,CREDIT.HISTORY.LENGTH_mon,branch_id_1,branch_id_2,branch_id_3,branch_id_5,branch_id_7,branch_id_8,branch_id_9,branch_id_10,branch_id_11,branch_id_13,branch_id_14,branch_id_15,branch_id_16,branch_id_17,branch_id_18,branch_id_19,branch_id_20,branch_id_29,branch_id_34,branch_id_35,branch_id_36,branch_id_42,branch_id_43,branch_id_48,branch_id_61,branch_id_62,branch_id_63,branch_id_64,branch_id_65,branch_id_66,branch_id_67,branch_id_68,branch_id_69,branch_id_70,branch_id_72,branch_id_73,branch_id_74,branch_id_76,branch_id_77,branch_id_78,branch_id_79,branch_id_82,branch_id_84,branch_id_85,branch_id_97,branch_id_100,branch_id_101,branch_id_103,branch_id_104,branch_id_105,branch_id_111,branch_id_117,branch_id_120,branch_id_121,branch_id_130,branch_id_135,branch_id_136,branch_id_138,branch_id_142,branch_id_146,branch_id_147,branch_id_152,branch_id_153,branch_id_158,branch_id_159,branch_id_160,branch_id_162,branch_id_165,branch_id_202,branch_id_207,branch_id_217,branch_id_248,branch_id_249,branch_id_250,branch_id_251,branch_id_254,branch_id_255,branch_id_257,branch_id_258,branch_id_259,branch_id_260,branch_id_261,manufacturer_id_45,manufacturer_id_48,manufacturer_id_49,manufacturer_id_51,manufacturer_id_67,manufacturer_id_86,manufacturer_id_120,manufacturer_id_145,manufacturer_id_152,manufacturer_id_153,manufacturer_id_156,Employment.Type_Salaried,Employment.Type_Self employed,Employment.Type_Unemployed,State_ID_1,State_ID_2,State_ID_3,State_ID_4,State_ID_5,State_ID_6,State_ID_7,State_ID_8,State_ID_9,State_ID_10,State_ID_11,State_ID_12,State_ID_13,State_ID_14,State_ID_15,State_ID_16,State_ID_17,State_ID_18,State_ID_19,State_ID_20,State_ID_21,State_ID_22,PERFORM_CNS.SCORE.DESCRIPTION_A-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_B-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_C-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_D-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_E-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_F-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_G-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_H-Medium Risk,PERFORM_CNS.SCORE.DESCRIPTION_I-Medium Risk,PERFORM_CNS.SCORE.DESCRIPTION_J-High Risk,PERFORM_CNS.SCORE.DESCRIPTION_K-High Risk,PERFORM_CNS.SCORE.DESCRIPTION_L-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_M-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_No Bureau History Available,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: More than 50 active Accounts found,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Activity seen on the customer (Inactive),PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Updates available in last 36 months,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Not Enough Info available on the customer,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Only a Guarantor,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Sufficient History Not Available
count,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05

### Training

#### Evaluation with Stratified kFold

In [100]:
skf = StratifiedKFold(n_splits=5)

results = []

for train_idx, test_idx in skf.split(X_train, y_train):
    model = LogisticRegression()
    model.fit(X_train.iloc[train_idx, :], y_train.iloc[train_idx])
    result = model.score(X_train.iloc[test_idx, :], y_train.iloc[test_idx])
    results.append(result)

In [101]:
for i, result in enumerate(results):
    print("Fold {}: {:.2f}%".format(i + 1, result * 100))

Fold 1: 78.24%
Fold 2: 78.22%
Fold 3: 77.84%
Fold 4: 78.05%
Fold 5: 74.57%


### Generating X_test predictions

We can assume our predictions are ~78% accurate.

In [102]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])